<a href="https://colab.research.google.com/github/FahimAhamed0602/assinment2-All-task/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

def initialize_population(population_size, chromosome_length):
    return [[random.randint(0, 1) for _ in range(chromosome_length)] for _ in range(population_size)]

def evaluate_fitness(chromosome, num_courses, num_timeslots):
    overlap_penalty = 0
    consistency_penalty = 0

    # Calculate overlap penalty
    for timeslot in range(num_timeslots):
        timeslot_courses = chromosome[timeslot * num_courses:(timeslot + 1) * num_courses]
        if sum(timeslot_courses) > 1:
            overlap_penalty += sum(timeslot_courses) - 1

    # Calculate consistency penalty
    course_counts = [0] * num_courses
    for timeslot in range(num_timeslots):
        timeslot_courses = chromosome[timeslot * num_courses:(timeslot + 1) * num_courses]
        for course in range(num_courses):
            course_counts[course] += timeslot_courses[course]

    for count in course_counts:
        consistency_penalty += abs(count - 1)

    total_penalty = overlap_penalty + consistency_penalty
    return -total_penalty  # Negative fitness for maximization

def select_parents(population, fitnesses):
    total_fitness = sum(fitnesses)
    parent1 = select_parent(population, fitnesses, total_fitness)
    parent2 = select_parent(population, fitnesses, total_fitness)
    return parent1, parent2

def select_parent(population, fitnesses, total_fitness):
    pick = random.uniform(0, total_fitness)
    current_fitness = 0
    for i, fitness in enumerate(fitnesses):
        current_fitness += fitness
        if current_fitness > pick:
            return population[i]
    return population[0]

def crossover_single_point(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 2)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]

def genetic_algorithm(num_courses, num_timeslots, population_size, mutation_rate, max_iterations):
    chromosome_length = num_courses * num_timeslots
    population = initialize_population(population_size, chromosome_length)
    best_chromosome = None
    best_fitness = float('-inf')

    for _ in range(max_iterations):
        fitnesses = [evaluate_fitness(chromosome, num_courses, num_timeslots) for chromosome in population]
        for i, fitness in enumerate(fitnesses):
            if fitness > best_fitness:
                best_fitness = fitness
                best_chromosome = population[i]

        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = select_parents(population, fitnesses)
            child1, child2 = crossover_single_point(parent1, parent2)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population[:population_size]

    return best_chromosome, best_fitness

def read_input_file(file_name):
    """Reads input from a file."""
    with open(file_name, 'r') as file:
        lines = file.readlines()
        num_courses, num_timeslots = map(int, lines[0].strip().split())
        courses = [line.strip() for line in lines[1:num_courses + 1]]
    return num_courses, num_timeslots, courses

def main():
    input_file = 'input.txt'
    num_courses, num_timeslots, courses = read_input_file(input_file)

    population_size = 10
    mutation_rate = 0.01
    max_iterations = 100

    best_chromosome, best_fitness = genetic_algorithm(num_courses, num_timeslots, population_size, mutation_rate, max_iterations)

    print("Best Chromosome:", ''.join(map(str, best_chromosome)))
    print("Best Fitness:", best_fitness)

if __name__ == "__main__":
    main()

Best Chromosome: 100000001
Best Fitness: -1


In [ ]:
# Function for two-point crossover
def two_point_crossover(parent1, parent2):
    point1 = random.randint(0, len(parent1) - 2)
    point2 = random.randint(point1 + 1, len(parent1) - 1)
    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
    return child1, child2

# Example usage of two-point crossover
parent1 = [0, 0, 0, 1, 1, 1, 0, 0, 0]
parent2 = [1, 1, 1, 0, 0, 0, 1, 1, 1]

child1, child2 = two_point_crossover(parent1, parent2)

print("Parent 1: ", ''.join(map(str, parent1)))
print("Parent 2: ", ''.join(map(str, parent2)))
print("Child 1:  ", ''.join(map(str, child1)))
print("Child 2:  ", ''.join(map(str, child2)))


Parent 1:  000111000
Parent 2:  111000111
Child 1:   000100000
Child 2:   111011111


In [ ]:
import random

# Function for tournament selection
def tournament_selection(population, fitnesses, k=3):
    selected = random.sample(list(zip(population, fitnesses)), k)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]

# Genetic Algorithm using Tournament Selection
def genetic_algorithm_tournament(N, T, courses, population_size, mutation_rate, max_iterations, tournament_size):
    chromosome_length = N * T
    population = generate_population(population_size, chromosome_length)
    best_chromosome = None
    best_fitness = float('-inf')

    for iteration in range(max_iterations):
        fitnesses = [fitness(chrom, N, T) for chrom in population]

        for i, fit in enumerate(fitnesses):
            if fit > best_fitness:
                best_fitness = fit
                best_chromosome = population[i]

        new_population = []
        while len(new_population) < population_size:
            parent1 = tournament_selection(population, fitnesses, tournament_size)
            parent2 = tournament_selection(population, fitnesses, tournament_size)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population

    return best_chromosome, best_fitness

# Function to read input from file
def read_input(file_name):
    with open(file_name, 'r') as file:
        N, T = map(int, file.readline().strip().split())
        courses = [line.strip() for line in file.readlines()]
    return N, T, courses

# Input file name
input_file = "input.txt"
N, T, courses = read_input(input_file)

# Parameters
population_size = 10
mutation_rate = 0.01
max_iterations = 100
tournament_size = 3

# Run Genetic Algorithm with Tournament Selection
best_chromosome, best_fitness = genetic_algorithm_tournament(N, T, courses, population_size, mutation_rate, max_iterations, tournament_size)

# Output
print("Best Chromosome:", ''.join(map(str, best_chromosome)))
print("Best Fitness:", best_fitness)


NameError: name 'generate_population' is not defined